In [5]:
import geojson
import json
import requests
import os
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
#setup JSONBin.io 
JSONBIN_ACCESS_KEY = '$2a$10$7B8tVje037JkjFrFjBuXauoSw5x7XdSVhSL2KfUqm7XfcNSOhQ8ae'
JSONBIN_API_URL = f'https://api.jsonbin.io/v3/b/66db5907acd3cb34a87f7d42'

# Fetch data from JSONBin
headers = {'X-Master-Key': JSONBIN_ACCESS_KEY}
response = requests.get(JSONBIN_API_URL, headers=headers)

if response.status_code == 200:
    json_data = response.json()
    # Print the structure for debugging
    print("JSON Data Structure:", json.dumps(json_data, indent=4))

    # Extract 'features' from JSON
    feature_list = json_data.get('features', [])
else:
    raise Exception(f"Failed to fetch data from JSONBin: {response.status_code}, {response.text}")

# Validate and inspect the feature list
if isinstance(feature_list, list):
    print("Data Sample:", feature_list[:2])
else:
    raise TypeError(f"Expected 'feature_list' to be a list, got {type(feature_list)}")

# Print data sample
print("Data Sample:", feature_list[:2])

JSON Data Structure: {
    "record": {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [
                        -77.56475249803376,
                        37.6374734793625
                    ]
                },
                "properties": {
                    "report_type": "Near Miss",
                    "event_type": "Collision2",
                    "travel_type": "Bicycle",
                    "injury_type": "No Inury",
                    "bhrs1": 1,
                    "bhrs2": 0,
                    "bhrs3": 0,
                    "bhrs4": 1,
                    "bhrs5": 0,
                    "bhrs6": 0,
                    "bhrs7": 0,
                    "bhrs8": 0,
                    "loc1": 0,
                    "loc2": 0,
                    "loc3": 0,
                    "loc4": 0,
                    "l

In [6]:
# Convert JSON to GeoDataFrame
features = json_data["record"]["features"]
geometry = [Point(feature["geometry"]["coordinates"]) for feature in features]

# Create GeoDataFrame with a specified CRS (WGS 84)
gdf = gpd.GeoDataFrame(features, geometry=geometry, crs="EPSG:4326")

# Save as a Shapefile
gdf.to_file("output.shp")

print("Shapefile created successfully with CRS EPSG:4326!")

Shapefile created successfully with CRS EPSG:4326!


c:\Users\egreenwell\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value '{'report_type': 'Near Miss', 'event_type': 'Collision2', 'travel_type': 'Bicycle', 'injury_type': 'No Inury', 'bhrs1': 1, 'bhrs2': 0, 'bhrs3': 0, 'bhrs4': 1, 'bhrs5': 0, 'bhrs6': 0, 'bhrs7': 0, 'bhrs8': 0, 'loc1': 0, 'loc2': 0, 'loc3': 0, 'loc4': 0, 'loc5': 0, 'loc6': 0, 'loc7': 0, 'loc8': 0, 'loc9': 0, 'loc10': 0, 'loc11': 0, 'loc12': 0, 'loc13': 0, 'text': 'Car did not give legal 3-foot ROW', 'datetime': '2024-09-15T13:49'}' of field properties has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


In [11]:
# Extract features
features = json_data["record"]["features"]

# Transform JSON data into a DataFrame
data = []
for feature in features:
    coordinates = feature["geometry"]["coordinates"]
    properties = feature["properties"]
    row = {
        "Longitude": coordinates[0],
        "Latitude": coordinates[1]
    }
    row.update(properties)
    data.append(row)

df = pd.DataFrame(data)

# Save to Excel
df.to_excel("output.xlsx", index=False)

print("Excel file 'output.xlsx' created successfully!")



Excel file 'output.xlsx' created successfully!


In [3]:
# ArcGIS Enterprise credentials and URL
portal_url = "https://planrva.maps.arcgis.com/home/index.html"  # Replace with your ArcGIS Enterprise URL
username = "egreenwell_PlanRVA"
password = "HOPPER!02isgreat"

# Authenticate with ArcGIS Enterprise
gis = GIS(portal_url, username, password)

# Step 1: Create an empty feature service
service_properties = {
    "title": "Near Miss Events",
    "serviceDescription": "Feature Service for near-miss events created from JSONBin data",
    "capabilities": "Create,Query,Update,Delete,Editing",  # Enable editing if needed
    "spatialReference": {"wkid": 4326},
    "description": "Feature Service created from JSONBin data",
}

# Create the service
feature_service = gis.content.create_service(name="Near Miss Events", service_type="featureService", create_params=service_properties)
print(f"Feature Service Created: {feature_service.title}")

# Step 2: Access the layers in the new service
feature_layer_collection = FeatureLayerCollection.fromitem(feature_service)

# Define layer properties
layer_properties = {
    "name": "Near Miss Events Layer",
    "geometryType": "esriGeometryPoint",
    "fields": [
        {"name": "report_type", "type": "esriFieldTypeString", "alias": "Report Type", "length": 255},
        {"name": "event_type", "type": "esriFieldTypeString", "alias": "Event Type", "length": 255},
        {"name": "travel_type", "type": "esriFieldTypeString", "alias": "Travel Type", "length": 255},
        {"name": "injury_type", "type": "esriFieldTypeString", "alias": "Injury Type", "length": 255},
        {"name": "datetime", "type": "esriFieldTypeDate", "alias": "Datetime"},
        {"name": "text", "type": "esriFieldTypeString", "alias": "Description", "length": 255},
    ],
    "spatialReference": {"wkid": 4326},
}

# Add a layer to the service
feature_layer_collection.manager.add_to_definition({"layers": [layer_properties]})
print(f"Layer added to Feature Service: {feature_layer_collection.layers[0].properties.name}")

# Step 3: Add features to the layer
feature_layer = feature_layer_collection.layers[0]

# Prepare features from the JSON data
features = []
for feature in feature_list:
    geometry = feature.get("geometry")
    attributes = feature.get("properties")
    if geometry and attributes:
        features.append({"geometry": geometry, "attributes": attributes})

# Add features to the layer
feature_layer.edit_features(adds=features)
print("Features successfully added to the layer.")

# Final output: Service URL
print(f"Feature Service URL: {feature_service.url}")

Exception: {'message': "Service name '' already exists for '7SazdyBTbp9w1Fga'"}